In [2]:
import tensorflow as tf
import os
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt

2023-07-03 08:21:12.472806: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 08:21:12.654390: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-03 08:21:12.655102: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-03 08:21:13.510740: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
IMAGE_SIZE = (224, 224)


In [4]:
model = tf.keras.applications.VGG16(
    include_top=True, weights= "imagenet", input_tensor=None, 
    input_shape=None, classes=1000
)

In [5]:
model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [6]:
model.save("VGG16_full_model.h5")

In [7]:
model.layers[1].padding

'same'

In [9]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

#create a directory
def create_dir(dir_name) :
    if not os.path.exists(dir_name) :
        os.makedirs(dir_name,exist_ok=True)
    print(f"{dir_name} directory created")

ROOT_DIR = "data"
create_dir(ROOT_DIR)

zip_file= "hymenoptera_data.zip"
zip_file_path = os.path.join(ROOT_DIR, zip_file)

if not os.path.isfile(zip_file):
    print("Downloading hymenoptera data")
    file_name, header = request.urlretrieve(data_URL, zip_file_path)
    print(f"Downloaded hymenoptera data")

else :
    print(f"data already exists")






data directory created
Downloaded hymenoptera data


In [10]:
from zipfile import ZipFile
zip_file_dir = "unzip_data_dir"
unzip_data_dir_path = os.path.join(ROOT_DIR, zip_file_dir)

if not os.path.exists(zip_file_dir) :
    os.makedirs(zip_file_dir, exist_ok=True)
    with ZipFile(zip_file_path) as unzip_data :
        unzip_data.extractall(unzip_data_dir_path)

else :
    print("unzip data already exists")

In [19]:
main_data_dir = Path("./data/unzip_data_dir/hymenoptera_data/train")
main_data_dir

PosixPath('data/unzip_data_dir/hymenoptera_data/train')

In [12]:
BATCH_SIZE = 32
pixels = 224
IMAGE_SIZE = (pixels, pixels)


In [14]:
data_kwargs = dict(rescale = 1./255, validation_split = 0.20)
dataflow_kwargs = dict(target_size = IMAGE_SIZE, batch_size = BATCH_SIZE)
data_kwargs 

{'rescale': 0.00392156862745098, 'validation_split': 0.2}

In [21]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_kwargs)

valid_datagenerator = valid_datagen.flow_from_directory(
    main_data_dir, subset = "validation", shuffle = False,
    **dataflow_kwargs)

do_data_augmentation = False

if do_data_augmentation :
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=40, zoom_range=0.2, horizontal_flip=True,
        width_shift_range=0.2, height_shift_range=0.2,
        shear_range=0.2,**data_kwargs
    )

else :
    train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(main_data_dir,
                                            subset="training",shuffle=True,
                                            **dataflow_kwargs)

Found 48 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [22]:
RGB_IMAGE_SIZE = (pixels, pixels,3)

vgg = tf.keras.applications.vgg16.VGG16(
    input_shape=RGB_IMAGE_SIZE,
    weights="imagenet",
    include_top=False
)

58889256/58889256 [==============================] - 99s 2us/step


In [23]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

input_2             : True
block1_conv1        : True
block1_conv2        : True
block1_pool         : True
block2_conv1        : True
block2_conv2        : True
block2_pool         : True
block3_conv1        : True
block3_conv2        : True
block3_conv3        : True
block3_pool         : True
block4_conv1        : True
block4_conv2        : True
block4_conv3        : True
block4_pool         : True
block5_conv1        : True
block5_conv2        : True
block5_conv3        : True
block5_pool         : True


In [25]:
for layer in vgg.layers:
    layer.trainable = False

In [26]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

input_2             : False
block1_conv1        : False
block1_conv2        : False
block1_pool         : False
block2_conv1        : False
block2_conv2        : False
block2_pool         : False
block3_conv1        : False
block3_conv2        : False
block3_conv3        : False
block3_pool         : False
block4_conv1        : False
block4_conv2        : False
block4_conv3        : False
block4_pool         : False
block5_conv1        : False
block5_conv2        : False
block5_conv3        : False
block5_pool         : False


In [34]:
vgg.inputs

[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>]

In [27]:
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [28]:
del model

In [31]:
CLASSES = 2
x = tf.keras.layers.Flatten()(vgg.output)
prediction = tf.keras.layers.Dense(CLASSES, activation="softmax")(x)

model = tf.keras.models.Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [32]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01,ema_momentum=8.9),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [33]:
EPOCHS = 10
history = model.fit(train_generator, epochs=EPOCHS, validation_data=valid_datagenerator)

Epoch 1/10


2023-07-03 10:25:42.075390: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - ETA: 0s - loss: 4.8465 - accuracy: 0.5867

2023-07-03 10:25:57.602629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


7/7 [==============================] - 19s 3s/step - loss: 4.8465 - accuracy: 0.5867 - val_loss: 1.6977 - val_accuracy: 0.8125
Epoch 2/10
7/7 [==============================] - 17s 2s/step - loss: 3.7703 - accuracy: 0.7449 - val_loss: 8.4951 - val_accuracy: 0.6250
Epoch 3/10
7/7 [==============================] - 18s 3s/step - loss: 2.2982 - accuracy: 0.8418 - val_loss: 3.1522 - val_accuracy: 0.7917
Epoch 4/10
7/7 [==============================] - 17s 2s/step - loss: 1.1752 - accuracy: 0.8878 - val_loss: 2.9247 - val_accuracy: 0.8333
Epoch 5/10
7/7 [==============================] - 17s 2s/step - loss: 0.5292 - accuracy: 0.9439 - val_loss: 2.3277 - val_accuracy: 0.8333
Epoch 6/10
7/7 [==============================] - 16s 2s/step - loss: 0.6242 - accuracy: 0.9388 - val_loss: 2.6269 - val_accuracy: 0.8333
Epoch 7/10
7/7 [==============================] - 16s 2s/step - loss: 0.0471 - accuracy: 0.9898 - val_loss: 3.6873 - val_accuracy: 0.7500
Epoch 8/10
7/7 [=============================